Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
# dropping 535 apartment observations that are in the above extremes
df.shape

(48817, 34)

In [4]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [6]:
df['created'] = pd.to_datetime(df['created'])

In [7]:
df['created'].loc[1]

Timestamp('2016-06-12 12:19:27')

In [8]:
train = df[df['created'] < '2016-06-01 00:00:00']

In [9]:
test = df[df['created'] >= '2016-06-01 00:00:00']

In [10]:
train.shape

(31844, 34)

In [11]:
test.shape

(16973, 34)

In [12]:
train['price'].mean()

3575.604007034292

In [13]:
target = 'price'
y_train = train[target]
y_test = test[target]

In [14]:
# Get mean baseline
guess = y_train.mean()

In [15]:
[guess] * len(y_train)

[3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,
 3575.604007034292,


In [16]:
# Train Error
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (April & May 2016) Apartments: ${mae:.2f} off')

Train Error (April & May 2016) Apartments: $1201.88 off


In [17]:
# Test Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error (June 2016) Apartments: ${mae:.2f} off')

Test Error (June 2016) Apartments: $1197.71 off


In [18]:
# Engineered Feature 1: 

df['rooms'] = df['bathrooms'] + df['bedrooms']

In [19]:
df['rooms'].sample(3)

30667    5.0
32177    2.0
6973     1.0
Name: rooms, dtype: float64

In [20]:
df['rooms'].mean()

2.738943400864453

In [21]:
# Engineered Feature 2: Total Perks per Apartment

perks = ["elevator", "cats_allowed", "hardwood_floors", "dogs_allowed", 
             "doorman", "dishwasher","no_fee","laundry_in_building", 
             "fitness_center","pre-war","laundry_in_unit", "roof_deck",
             "outdoor_space","dining_room","high_speed_internet", 
             "balcony", "swimming_pool", "new_construction", "terrace", 
             "exclusive", "loft", "garden_patio", "wheelchair_access", 
             "common_outdoor_space"]

def get_total_perks(cell):
    result = 0
    for perk in perks:
        if cell[perk]:
            result += 1
    return result

df['total_perks'] = df.apply(get_total_perks, axis=1)

In [22]:
df['total_perks'].sample(5)

7022     2
8764     5
27047    4
12241    4
32412    3
Name: total_perks, dtype: int64

In [23]:
df['total_perks'].mean()

4.672593563717557

In [24]:
train = df[df['created'] < '2016-06-01 00:00:00']

In [25]:
test = df[df['created'] >= '2016-06-01 00:00:00']

In [26]:
# fitting models to 2 engineered features
features = ['rooms', 'total_perks']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

print('Shape of train:', X_train.shape)
print('Shape of test:', X_test.shape)

Shape of train: (31844, 2)
Shape of test: (16973, 2)


In [27]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [28]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def get_metrics(y_pred, y_real):
    print("RMSE:", np.sqrt(mean_squared_error(y_pred, y_real)))
    print("MAE:", mean_absolute_error(y_pred, y_real))
    print("R^2:", r2_score(y_pred, y_real))

In [30]:
# fitting model and checking metrics on training data
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
get_metrics(y_pred, y_train)

RMSE: 1295.7149658677172
MAE: 856.6168448694106
R^2: -0.17720868083501884


In [31]:
# checking metrics on test data 
y_pred = model.predict(X_test)
get_metrics(y_pred, y_test)

RMSE: 1293.2257015977382
MAE: 867.9967359241994
R^2: -0.17900573134238762


In [32]:
model.intercept_, model.coef_

(998.1946341143803, array([767.95662272, 101.76435155]))

In [33]:
beta0 = model.intercept_
beta1, beta2 = model.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

y = 998.1946341143803 + 767.9566227199571x1 + 101.76435154698719x2


In [34]:
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept 998.1946341143803
rooms          767.956623
total_perks    101.764352
